In [14]:
# 1B) run HCopy
import subprocess, shlex
cmd = ["HCopy","-C","param.cfg","-S","mfcc.lst"]
print("▶", " ".join(map(shlex.quote, cmd)))
subprocess.run(cmd, check=True)


▶ HCopy -C param.cfg -S mfcc.lst


CompletedProcess(args=['HCopy', '-C', 'param.cfg', '-S', 'mfcc.lst'], returncode=0)

In [15]:
# 2A) prot.lst = second column of mfcc.lst
from pathlib import Path
scp = Path("mfcc.lst"); out = Path("prot/prot.lst"); out.parent.mkdir(parents=True, exist_ok=True)
mfc_lines = []
for line in scp.read_text(encoding="utf-8").splitlines():
    if not line.strip(): continue
    mfc = line.split()[1]
    mfc_lines.append(mfc+"\n")
out.write_text("".join(mfc_lines), encoding="utf-8")
print("Wrote", out, "with", len(mfc_lines), "paths")


Wrote prot/prot.lst with 7521 paths


In [16]:
# 2B) HCompV: flat start (global mean/var, vFloors)
import subprocess, shlex
cmd = ["HCompV","-C","prot/config","-f","0.01","-m","-S","prot/prot.lst","-M","hmm","prot/prot"]
print("▶", " ".join(map(shlex.quote, cmd)))
subprocess.run(cmd, check=True)
# outputs: hmm/prot and hmm/vFloors


▶ HCompV -C prot/config -f 0.01 -m -S prot/prot.lst -M hmm prot/prot


CompletedProcess(args=['HCompV', '-C', 'prot/config', '-f', '0.01', '-m', '-S', 'prot/prot.lst', '-M', 'hmm', 'prot/prot'], returncode=0)

In [20]:
# 3) create hmm/hmm.0/{macros,hmmdefs}
from pathlib import Path
hmmDir = Path("hmm"); proto = hmmDir/"prot"; vf = hmmDir/"vFloors"
mono = Path("monophones.lst")
hmm0 = hmmDir/"hmm.0"; hmm0.mkdir(parents=True, exist_ok=True)
macros0 = hmm0/"macros"; hmmdefs0 = hmm0/"hmmdefs"

lines = proto.read_text(errors="ignore").splitlines()

# extract the exact ~o header block from hmm/prot
o_start = next((i for i,l in enumerate(lines) if l.strip().startswith("~o")), None)
assert o_start is not None, "HCompV proto has no ~o"
header = [lines[o_start]]
for ln in lines[o_start+1:]:
    s = ln.strip()
    if s.startswith("~") or s.lower().startswith("<beginhmm>"): break
    header.append(ln)
#assert any("<VecSize>" in h for h in header), "~o lacks <VecSize>"

# write macros: ~o then vFloors
macros0.write_text("\n".join(header)+"\n"+vf.read_text(errors="ignore"), encoding="utf-8")

# clone the HMM body for each phone
b = next(i for i,l in enumerate(lines) if l.strip().lower().startswith("<beginhmm>"))
body = "\n".join(lines[b:])+"\n"
phones = [p.strip() for p in mono.read_text(encoding="utf-8").splitlines() if p.strip()]
with hmmdefs0.open("w", encoding="utf-8") as f:
    for ph in phones:
        f.write(f"~h \"{ph}\"\n{body}\n")
print("Built", macros0, "and", hmmdefs0, "phones:", len(phones))


Built hmm/hmm.0/macros and hmm/hmm.0/hmmdefs phones: 48


In [22]:
import subprocess as sp, shlex
from pathlib import Path

def run(cmd):
    print("▶", " ".join(map(shlex.quote, cmd)))
    p = sp.run(cmd, stdout=sp.PIPE, stderr=sp.PIPE, text=True)
    if p.stdout: print(p.stdout)
    if p.returncode:
        print(p.stderr)
        raise SystemExit(f"cmd failed (exit {p.returncode})")

# paths
mlf   = "train_phones.mlf"
scp   = "prot/prot.lst"
cfg   = "hmm/config"
mono  = "monophones.lst"

# 1) a couple of HERest passes at 1 mixture
prev = Path("hmm/hmm.0")
for k in (1, 2):  # do more (e.g., 3–5) if you want
    out = Path(f"hmm/hmm.{k}"); out.mkdir(parents=True, exist_ok=True)
    run(["HERest", "-t","250.0","150.0","1000.0",
         "-C", cfg, "-I", mlf, "-S", scp,
         "-H", str(prev/"macros"), "-H", str(prev/"hmmdefs"),
         "-M", str(out), mono])
    prev = out

# 2) mixture growth: 2 → 4 → 8 → 16, 2× HERest after each MU
def write_mu_hed(n, path):
    path.write_text(f"MU {n} {{*.state[2-4].mix}}\n", encoding="utf-8")

current = prev  # start from last 1-mixture pass
for nmix in [2, 4, 8, 16]:
    hed = Path(f"hmm/com{nmix}mix.hed"); write_mu_hed(nmix, hed)
    split_dir = Path(f"hmm/mono{nmix}.0"); split_dir.mkdir(parents=True, exist_ok=True)
    # split mixtures
    run(["HHEd", "-H", str(current/"hmmdefs"), "-M", str(split_dir), str(hed), mono])
    # two HERest iters after split (per slides)
    it1 = Path(f"hmm/mono{nmix}.1"); it1.mkdir(exist_ok=True)
    run(["HERest", "-t","250.0","150.0","1000.0",
         "-C", cfg, "-I", mlf, "-S", scp,
         "-H", str(split_dir/"hmmdefs"), "-M", str(it1), mono])
    it2 = Path(f"hmm/mono{nmix}.2"); it2.mkdir(exist_ok=True)
    run(["HERest", "-t","250.0","150.0","1000.0",
         "-C", cfg, "-I", mlf, "-S", scp,
         "-H", str(it1/"hmmdefs"), "-M", str(it2), mono])
    current = it2
    print(f"→ Reached {nmix} mixtures: {current}")

print("Final multi-mixture hmmdefs:", current/"hmmdefs")


▶ HERest -t 250.0 150.0 1000.0 -C hmm/config -I train_phones.mlf -S prot/prot.lst -H hmm/hmm.0/macros -H hmm/hmm.0/hmmdefs -M hmm/hmm.1 monophones.lst
Pruning-On[250.0 150.0 1000.0]
 WARNING [-2331]  UpdateModels: n0[30] copied: only 0 egs
 in HERest
 WARNING [-2331]  UpdateModels: n1[31] copied: only 0 egs
 in HERest
 WARNING [-2331]  UpdateModels: n2[32] copied: only 0 egs
 in HERest
 WARNING [-2331]  UpdateModels: n3[33] copied: only 0 egs
 in HERest
 WARNING [-2331]  UpdateModels: n4[34] copied: only 0 egs
 in HERest
 WARNING [-2331]  UpdateModels: n5[35] copied: only 0 egs
 in HERest

▶ HERest -t 250.0 150.0 1000.0 -C hmm/config -I train_phones.mlf -S prot/prot.lst -H hmm/hmm.1/macros -H hmm/hmm.1/hmmdefs -M hmm/hmm.2 monophones.lst
Pruning-On[250.0 150.0 1000.0]
 WARNING [-2331]  UpdateModels: n0[30] copied: only 0 egs
 in HERest
 WARNING [-2331]  UpdateModels: n1[31] copied: only 0 egs
 in HERest
 WARNING [-2331]  UpdateModels: n2[32] copied: only 0 egs
 in HERest
 WARNING [-233

In [ ]:
# import subprocess as sp, shlex
from pathlib import Path
import re

# ---- paths/configs ----
cfg  = "hmm/config"
mlf  = "train_phones.mlf"
scp  = "prot/prot.lst"
mono = "monophones.lst"

# which mixture families to extend
mixtures = [2, 4, 8, 16, 32]  # adjust as you like

def run(cmd):
    print("▶", " ".join(map(shlex.quote, cmd)))
    p = sp.run(cmd, stdout=sp.PIPE, stderr=sp.PIPE, text=True)
    if p.stdout: print(p.stdout)
    if p.returncode:
        print(p.stderr)
        raise SystemExit(f"cmd failed (exit {p.returncode})")

def newest_iter_dir(base_dir: Path) -> int:
    """
    Return the highest iteration number N such that base_dir.parent / f"{base_dir.name_root}.{N}"
    contains an hmmdefs file. Example base_dir for mono8 is Path('hmm/mono8').
    For the 1-mixture family, base_dir is Path('hmm/hmm').
    """
    parent = base_dir.parent
    stem   = base_dir.name
    pattern = re.compile(re.escape(stem) + r"\.(\d+)$")
    latest = 0
    for p in parent.glob(stem + ".*"):
        if p.is_dir():
            m = pattern.match(p.name)
            if m:
                it = int(m.group(1))
                if (p/"hmmdefs").exists():
                    latest = max(latest, it)
    return latest

def herest_iter(prev_dir: Path, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    # For subsequent iterations we only need the previous hmmdefs (macros already “frozen”)
    run(["HERest", "-t","250.0","150.0","1000.0",
         "-C", cfg, "-I", mlf, "-S", scp,
         "-H", str(prev_dir/"hmmdefs"),
         "-M", str(out_dir),
         mono])

# -------- 1) extend the 1-mixture family: hmm/hmm.N --------
base_hmm = Path("hmm/hmm")
cur_it = newest_iter_dir(base_hmm)  # e.g., 2
if cur_it == 0:
    print("No 1-mixture family found (hmm/hmm.N). Skipping.")
else:
    print(f"[1-mixture] starting at iteration {cur_it}")
    while cur_it < 6:
        nxt = cur_it + 1
        prev_dir = Path(f"hmm/hmm.{cur_it}")
        out_dir  = Path(f"hmm/hmm.{nxt}")
        print(f"→ HERest {prev_dir.name} → {out_dir.name}")
        herest_iter(prev_dir, out_dir)
        cur_it = nxt
    print(f"[1-mixture] reached iteration {cur_it}")

# -------- 2) extend each mixture family: hmm/mono{M}.N --------
for m in mixtures:
    base = Path(f"hmm/mono{m}")
    cur_it = newest_iter_dir(base)
    if cur_it == 0:
        print(f"[mono{m}] not found (no mono{m}.N with hmmdefs). Skipping.")
        continue
    print(f"[mono{m}] starting at iteration {cur_it}")
    while cur_it < 6:
        nxt = cur_it + 1
        prev_dir = Path(f"hmm/mono{m}.{cur_it}")
        out_dir  = Path(f"hmm/mono{m}.{nxt}")
        print(f"→ HERest {prev_dir.name} → {out_dir.name}")
        herest_iter(prev_dir, out_dir)
        cur_it = nxt
    print(f"[mono{m}] reached iteration {cur_it}")

print("Done.")


In [3]:
from pathlib import Path
import subprocess as sp, shlex, sys, re

root = Path(".")
mono = root/"monophones.lst"
hmmdefs = root/"hmm/mono8.6/hmmdefs"  # change if testing a different set
cfg = "hmm/config"
test_scp = root/"test.scp"            # must already exist

txt = hmmdefs.read_text(errors="ignore")
names = re.findall(r'~h\s+"([^"]+)"', txt)
print("Total models:", len(names))
print("Has si?:", "si" in names, " Has sil?:", "sil" in names)
print("First 10 names:", sorted(names))
print("Last 10 names:",  sorted(names))


def run(cmd):
    print("▶", " ".join(map(shlex.quote, cmd)))
    p = sp.run(cmd, stdout=sp.PIPE, stderr=sp.STDOUT, text=True)
    print(p.stdout)
    if p.returncode:
        sys.exit(f"cmd failed (exit {p.returncode})")
    return p

coding = "cp1250" #""utf-8"

# --- load phones & pick silence ---
phones = [p.strip() for p in mono.read_text(encoding=coding).splitlines() if p.strip()]
sil = "si" if "si" in phones else ("sil" if "sil" in phones else None)
if sil is None:
    sys.exit("No silence model found in monophones.lst (need 'si' or 'sil').")

# --- build loop set (exclude non-speech & the silence token) ---
exclude_ns = {"n0","n1","n2","n3","n4","n5"}
loop_phones = [p for p in phones if p not in exclude_ns and p != sil]

# --- (2) dictionary (ASCII-ascending) ---
dic = root/"phones.dic"
phones_sorted = sorted(phones)
dic.write_text("\n".join(f"{p}\t{p}" for p in phones_sorted) + "\n", encoding=coding)

# --- (2b) word list (ASCII-ascending) ---
wlist = root/"phones.wlist"
loop_phones_sorted = sorted(loop_phones)
wlist.write_text("\n".join(loop_phones_sorted + [sil]) + "\n", encoding=coding)

# --- (3) grammar (sil – one-or-more phones – sil), alternatives sorted ---
gram = root/"phoneloop.gram"
alts = " | ".join(loop_phones_sorted)
gram.write_text(f"$P = ( {alts} );\n( {sil} < $P > {sil} )\n", encoding=coding)

# --- (4) sanity-check silence model exists in hmmdefs ---
#hmm_text = hmmdefs.read_text(errors="ignore")
#if not re.search(rf'~h\s+"{re.escape(sil)}"\b', hmm_text):
#    sys.exit(f'HMM set {hmmdefs} does not contain ~h "{sil}" (silence)')

# --- (5) compile network & decode ---
wdnet = root/"phoneloop.slf"
run(["HParse", str(gram), str(wdnet)])

run([
  "HVite",
  "-T","1",
  "-C","hmm/config",
  "-H", str(hmmdefs),      # your model set
  "-S","test.scp",         # list of .mfcc files
  "-i","reco_mono8.6.mlf",
  "-w","phoneloop.slf",    # compiled network
  "phones.dic",            # dictFile
  "monophones.lst"         # hmmList  (or use hmmlist_ci.map if you built a mapping)
])


Total models: 48
Has si?: True  Has sil?: False
First 10 names: ['a', 'aa', 'b', 'ch', 'd', 'dg', 'dj', 'dz', 'e', 'ee', 'f', 'g', 'h', 'i', 'ii', 'k', 'l', 'm', 'mg', 'n', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'ng', 'nj', 'o', 'oo', 'p', 'r', 'rs', 'rz', 's', 'sh', 'si', 'swa', 't', 'tj', 'ts', 'u', 'uu', 'v', 'x', 'y', 'z', 'zh']
Last 10 names: ['a', 'aa', 'b', 'ch', 'd', 'dg', 'dj', 'dz', 'e', 'ee', 'f', 'g', 'h', 'i', 'ii', 'k', 'l', 'm', 'mg', 'n', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'ng', 'nj', 'o', 'oo', 'p', 'r', 'rs', 'rz', 's', 'sh', 'si', 'swa', 't', 'tj', 'ts', 'u', 'uu', 'v', 'x', 'y', 'z', 'zh']
▶ HParse phoneloop.gram phoneloop.slf

▶ HVite -T 1 -C hmm/config -H hmm/mono8.6/hmmdefs -S test.scp -i reco_mono8.6.mlf -w phoneloop.slf phones.dic monophones.lst
Read 48 physical / 48 logical HMMs
Read lattice with 46 nodes / 126 arcs
Created network with 91 nodes / 171 links
File: data/Test/ZKF/001.mfcc
si f k a d aa m si  ==  [198 frames] -58.2153 [Ac=-11526.6 LM=0.0] (Act=87.4)

CompletedProcess(args=['HVite', '-T', '1', '-C', 'hmm/config', '-H', 'hmm/mono8.6/hmmdefs', '-S', 'test.scp', '-i', 'reco_mono8.6.mlf', '-w', 'phoneloop.slf', 'phones.dic', 'monophones.lst'], returncode=0, stdout='Read 48 physical / 48 logical HMMs\nRead lattice with 46 nodes / 126 arcs\nCreated network with 91 nodes / 171 links\nFile: data/Test/ZKF/001.mfcc\nsi f k a d aa m si  ==  [198 frames] -58.2153 [Ac=-11526.6 LM=0.0] (Act=87.4)\nFile: data/Test/ZKF/002.mfcc\nsi n b o y dg e n aa ng si  ==  [198 frames] -59.7341 [Ac=-11827.3 LM=0.0] (Act=87.4)\nFile: data/Test/ZKF/003.mfcc\nsi p sh e r y oo u si  ==  [198 frames] -57.3579 [Ac=-11356.9 LM=0.0] (Act=87.4)\nFile: data/Test/ZKF/004.mfcc\nsi n ng d aa d i t e si  ==  [198 frames] -58.8171 [Ac=-11645.8 LM=0.0] (Act=87.4)\nFile: data/Test/ZKF/005.mfcc\nsi k e m y oo u si  ==  [198 frames] -59.9729 [Ac=-11874.6 LM=0.0] (Act=87.4)\nFile: data/Test/ZKF/006.mfcc\nsi f t f r ee y ng d i sh e i k f si  ==  [198 frames] -64.0168 [Ac=-12675.3 

In [7]:
from pathlib import Path
import subprocess as sp, shlex, re, sys

root = Path(".")
ref_mlf = root/"test_phones.mlf"
wlist   = root/"phones.wlist"      # phones to score (sorted is fine)
# reco files you produced with HVite:
reco_mlfs = [p for p in root.glob("reco_*.mlf") if p.is_file()]

if not reco_mlfs:
    sys.exit("No reco_*.mlf found. Run HVite first.")

def run(cmd):
    print("▶", " ".join(map(shlex.quote, cmd)))
    p = sp.run(cmd, stdout=sp.PIPE, stderr=sp.STDOUT, text=True)
    out = p.stdout
    print(out)
    if p.returncode:
        sys.exit(f"cmd failed (exit {p.returncode})")
    return out

def score(reco):
    # ignore silence/non-speech labels – adjust set to your symbols
    ignore_opts = ["-e","???","si","-e","???","n0","-e","???","n1",
                   "-e","???","n2","-e","???","n3","-e","???","n4","-e","???","n5"]
    cmd = ["HResults","-p","-s","-I", str(ref_mlf), *ignore_opts, str(wlist), str(reco)]
    out = run(cmd)
    # extract Acc or NIST-style summary line
    m = re.search(r"WORD:\s+%Corr=.*Acc=([0-9.]+)", out) or \
        re.search(r"\|\s+Sum/Avg\s+\|\s+\d+\s+\|\s+[0-9.]+\s+[0-9.]+\s+[0-9.]+\s+([0-9.]+)", out)
    acc = float(m.group(1)) if m else None
    return acc, out

print("\n===== HResults summary =====")
for reco in sorted(reco_mlfs):
    acc, _ = score(reco)
    tag = reco.stem.replace("reco_","")
    print(f"{tag:>16}: Acc={acc:.2f}%" if acc is not None else f"{tag:>16}: (see output)")
print("============================\n")



===== HResults summary =====
▶ HResults -p -s -I test_phones.mlf -e '???' si -e '???' n0 -e '???' n1 -e '???' n2 -e '???' n3 -e '???' n4 -e '???' n5 phones.wlist reco_hmm_1.mlf
====================== HTK Results Analysis =======================
  Date: Mon Oct 20 19:16:22 2025
  Ref : test_phones.mlf
  Rec : reco_hmm_1.mlf
------------------------ Overall Results --------------------------
SENT: %Correct=0.00 [H=0, S=192, N=192]
WORD: %Corr=7.65, Acc=-2.15 [H=82, D=237, S=753, I=105, N=1072]
------------------------ Confusion Matrix -------------------------
       a   a   b   c   d   d   d   d   e   f   g   h   i   k   m   m   n   n   n   o   o   p   r   s   s   s   t   t   u   x   y   z   z 
           a       h       g   j   z   e               i           g       g   j       o       s       h   w       j   u               h 
                                                                                                           a                              Del [ %c / %e]
   a 

In [1]:
from pathlib import Path
import subprocess as sp, shlex, sys, re

root = Path(".")
mono = root/"monophones.lst"
cfg = "hmm/config"
test_scp = root/"test.scp"            # must already exist
coding = "cp1250"                     # your project encoding

def run_quiet(cmd):
    # run a command quietly; surface stderr if it fails
    p = sp.run(cmd, stdout=sp.PIPE, stderr=sp.PIPE, text=True)
    if p.returncode:
        raise SystemExit(f"cmd failed (exit {p.returncode}): {' '.join(map(shlex.quote, cmd))}\n{p.stderr}")
    return p.stdout

# ---- collect all model directories to test (those that have hmmdefs), skipping hmm/hmm.0 ----
model_dirs = []
model_dirs += sorted((root/"hmm").glob("hmm.*"))
model_dirs += sorted((root/"hmm").glob("mono*.*"))
model_dirs = [d for d in model_dirs if (d/"hmmdefs").exists()
              and not (d.parent.name == "hmm" and d.name == "hmm.0")]
if not model_dirs:
    sys.exit("No model directories with hmmdefs found under hmm/")

# ---- common phone assets from monophones.lst ----
phones = [p.strip() for p in mono.read_text(encoding=coding).splitlines() if p.strip()]
sil = "si" if "si" in phones else ("sil" if "sil" in phones else None)
if sil is None:
    sys.exit("No silence model found in monophones.lst (need 'si' or 'sil').")
exclude_ns = {"n0","n1","n2","n3","n4","n5"}
loop_phones = [p for p in phones if p not in exclude_ns and p != sil]
phones_sorted = sorted(phones)
loop_phones_sorted = sorted(loop_phones)

# quick checks
if not test_scp.exists():
    sys.exit("Missing test.scp — build it from data/Test/**/*.mfcc first.")
ref_mlf = root/"test_phones.mlf"
if not ref_mlf.exists():
    sys.exit("Missing test_phones.mlf — build the reference MLF for Test first.")

# Acc results
acc_summary = []

# === LOOP: build grammar/dict, compile, decode, then score for each model set ===
for model_dir in model_dirs:
    hmmdefs = model_dir/"hmmdefs"
    tag = model_dir.name.replace(".", "_")  # e.g., mono8_6

    # (2) dictionary (ASCII-ascending)
    dic = root/"phones.dic"
    dic.write_text("\n".join(f"{p}\t{p}" for p in phones_sorted) + "\n", encoding=coding)

    # (2b) word list (ASCII-ascending)
    wlist = root/"phones.wlist"
    wlist.write_text("\n".join(loop_phones_sorted + [sil]) + "\n", encoding=coding)

    # (3) grammar (sil – one-or-more phones – sil), alternatives sorted
    gram = root/"phoneloop.gram"
    alts = " | ".join(loop_phones_sorted)
    gram.write_text(f"$P = ( {alts} );\n( {sil} < $P > {sil} )\n", encoding=coding)

    # compile network (quiet)
    wdnet = root/f"phoneloop_{tag}.slf"
    run_quiet(["HParse", str(gram), str(wdnet)])

    # decode (quiet)
    reco_path = root/f"reco_{tag}.mlf"
    run_quiet([
      "HVite",
      "-T","1",
      "-C", cfg,
      "-H", str(hmmdefs),
      "-S", str(test_scp),
      "-i", str(reco_path),
      "-w", str(wdnet),
      "phones.dic",
      "monophones.lst"
    ])

    # score (quiet) and extract Acc
    ignore_opts = ["-e","???","si","-e","???","n0","-e","???","n1",
                   "-e","???","n2","-e","???","n3","-e","???","n4","-e","???","n5"]
    out = run_quiet(["HResults","-p","-s","-I", str(ref_mlf), *ignore_opts, str(wlist), str(reco_path)])
    m = re.search(r"WORD:\s+%Corr=.*Acc=([0-9.]+)", out) or \
        re.search(r"\|\s+Sum/Avg\s+\|\s+\d+\s+\|\s+[0-9.]+\s+[0-9.]+\s+[0-9.]+\s+([0-9.]+)", out)
    acc = float(m.group(1)) if m else float("nan")
    if acc is float("nan"):
        print(f"[debug] No Acc parsed for {reco.name}; raw HResults output below:")
        print(out)
        #acc = float("nan")
    acc_summary.append((tag, acc))

# print only accuracies
for tag, acc in sorted(acc_summary, key=lambda x: x[0]):
    print(f"{tag}: {acc:.2f}%")


SystemExit: cmd failed (exit 3): HResults -p -s -I test_phones.mlf -e '???' si -e '???' n0 -e '???' n1 -e '???' n2 -e '???' n3 -e '???' n4 -e '???' n5 phones.wlist reco_hmm_1.mlf
  ERROR [+3331]  Index: Label adam- not in list[0 of 42]
 FATAL ERROR - Terminating program HResults


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import subprocess as sp, shlex, re
from pathlib import Path

def run(cmd):
    p = sp.run(cmd, stdout=sp.PIPE, stderr=sp.STDOUT, text=True)
    return p.stdout, p.returncode

def score(reco):
    ign = ["-e","???","si","-e","???","n0","-e","???","n1","-e","???","n2","-e","???","n3","-e","???","n4","-e","???","n5"]
    cmd = ["HResults","-p","-s","-I","test_phones.mlf", *ign, "phones.wlist", str(reco)]
    out, rc = run(cmd); 
    m = re.search(r"WORD:\s+%Corr=.*Acc=([0-9.]+)", out)
    return float(m.group(1)) if m else float("nan")

hmmdefs = "hmm/mono16.6/hmmdefs"  # use your best set
pairs = [(p,s) for p in (-16, -18, -20, -22, -24) for s in (13, 14, 15, 16, 17)]
rows = []
for p,s in pairs:
    tag = f"p{p}_s{s}"
    reco = Path(f"reco_mono16.6_{tag}.mlf")
    hv_cmd = ["HVite","-C","hmm/config","-H",hmmdefs,"-S","test.scp",
              "-i",str(reco),"-w","phoneloop.slf","-p",str(p),"-s",str(s),
              "phones.dic","monophones.lst"]
    _, rc = run(hv_cmd)
    acc = score(reco)
    rows.append((acc,p,s))
for acc,p,s in sorted(rows, reverse=True):
    print(f"{acc:6.2f}%   -p {p:>3}   -s {s}")


   nan%   -p -16   -s 13
   nan%   -p -16   -s 14
   nan%   -p -16   -s 15
   nan%   -p -16   -s 16
   nan%   -p -16   -s 17
   nan%   -p -18   -s 13
   nan%   -p -18   -s 14
   nan%   -p -18   -s 15
   nan%   -p -18   -s 16
   nan%   -p -18   -s 17
   nan%   -p -20   -s 13
   nan%   -p -20   -s 14
   nan%   -p -20   -s 15
   nan%   -p -20   -s 16
   nan%   -p -20   -s 17
   nan%   -p -22   -s 13
   nan%   -p -22   -s 14
   nan%   -p -22   -s 15
   nan%   -p -22   -s 16
   nan%   -p -22   -s 17
   nan%   -p -24   -s 13
   nan%   -p -24   -s 14
   nan%   -p -24   -s 15
   nan%   -p -24   -s 16
   nan%   -p -24   -s 17


blind shooting:

38.81%   -p -20   -s 15
 38.81%   -p -20   -s 12
 38.81%   -p -20   -s 10
 38.81%   -p -20   -s 5
 36.94%   -p -30   -s 15
 36.94%   -p -30   -s 12
 36.94%   -p -30   -s 10
 36.94%   -p -30   -s 5
 33.68%   -p -40   -s 15
 33.68%   -p -40   -s 12
 33.68%   -p -40   -s 10
 33.68%   -p -40   -s 5
 32.84%   -p -10   -s 15
 32.84%   -p -10   -s 12
 32.84%   -p -10   -s 10
 32.84%   -p -10   -s 5
 28.36%   -p -60   -s 15
 28.36%   -p -60   -s 12
 28.36%   -p -60   -s 10
 28.36%   -p -60   -s 5
 22.29%   -p -80   -s 15
 22.29%   -p -80   -s 12
 22.29%   -p -80   -s 10
 22.29%   -p -80   -s 5

 finding the sweetspot:
 38.90%   -p -22   -s 17
 38.90%   -p -22   -s 16
 38.90%   -p -22   -s 15
 38.90%   -p -22   -s 14
 38.90%   -p -22   -s 13

In [4]:
import subprocess as sp, shlex, re
from pathlib import Path

def run(cmd):
    p = sp.run(cmd, stdout=sp.PIPE, stderr=sp.STDOUT, text=True)
    return p.stdout, p.returncode

def score(reco):
    ign = ["-e","???","si","-e","???","n0","-e","???","n1","-e","???","n2","-e","???","n3","-e","???","n4","-e","???","n5"]
    cmd = ["HResults","-p","-s","-I","test_phones.mlf", *ign, "phones.wlist", str(reco)]
    out, rc = run(cmd)
    m = re.search(r"WORD:\s+%Corr=.*Acc=([0-9.]+)", out)
    return float(m.group(1)) if m else float("nan")

hmmdefs = "hmm/mono16.6/hmmdefs"  # use your best set

# grids to try
pairs = [(p,s) for p in (-20, -30) for s in (15,12)]
ts    = [(250,150,1000), (250,150,2000), (300,200,2000), (200,100,1000), (300,150,1000)]

rows = []
for (p,s) in pairs:
    for t in ts:
        ttag = f"t{t[0]}_{t[1]}_{t[2]}"
        tag  = f"p{p}_s{s}_{ttag}"
        reco = Path(f"reco_mono16.6_{tag}.mlf")
        hv_cmd = [
            "HVite","-C","hmm/config",
            "-t", str(t[0]), str(t[1]), str(t[2]),   # ← pruning beams
            "-H",hmmdefs,"-S","test.scp",
            "-i",str(reco),"-w","phoneloop.slf",
            "-p",str(p),"-s",str(s),
            "phones.dic","monophones.lst"
        ]
        _, rc = run(hv_cmd)
        acc = score(reco)
        rows.append((acc,p,s,t))

for acc,p,s,t in sorted(rows, reverse=True, key=lambda x: (x[0],)):
    print(f"{acc:6.2f}%   -p {p:>3}   -s {s}   -t {t[0]} {t[1]} {t[2]}")

# nema vliv

   nan%   -p -20   -s 15   -t 250 150 1000
   nan%   -p -20   -s 15   -t 250 150 2000
   nan%   -p -20   -s 15   -t 300 200 2000
   nan%   -p -20   -s 15   -t 200 100 1000
   nan%   -p -20   -s 15   -t 300 150 1000
   nan%   -p -20   -s 12   -t 250 150 1000
   nan%   -p -20   -s 12   -t 250 150 2000
   nan%   -p -20   -s 12   -t 300 200 2000
   nan%   -p -20   -s 12   -t 200 100 1000
   nan%   -p -20   -s 12   -t 300 150 1000
   nan%   -p -30   -s 15   -t 250 150 1000
   nan%   -p -30   -s 15   -t 250 150 2000
   nan%   -p -30   -s 15   -t 300 200 2000
   nan%   -p -30   -s 15   -t 200 100 1000
   nan%   -p -30   -s 15   -t 300 150 1000
   nan%   -p -30   -s 12   -t 250 150 1000
   nan%   -p -30   -s 12   -t 250 150 2000
   nan%   -p -30   -s 12   -t 300 200 2000
   nan%   -p -30   -s 12   -t 200 100 1000
   nan%   -p -30   -s 12   -t 300 150 1000
